In [2]:
import requests
from requests import Response
url = "https://tpnco.blob.core.windows.net/blobfs/Rally/TodayUrgentCase.json"
try:
    response:Response = requests.get(url)
    response_json = response.json()
    print(response.text)
except Exception as e:
    print(e)


{"type": "FeatureCollection","features": [
{"type": "Feature",	"geometry":{"type": "Polygon",	"coordinates": [[[302384.879,2778195.38],[302378.1,2778192.502],[302399.773,2778152.67],[302465.993,2778073.079],[302472.266,2778079.746],[302471.703,2778079.602],[302471.124,2778079.546],[302470.162,2778079.56],[302469.004,2778079.716],[302467.713,2778079.963],[302466.62,2778080.328],[302465.676,2778080.851],[302464.849,2778081.409],[302463.377,2778082.788],[302446.355,2778103.631],[302419.356,2778136.607],[302408.458,2778150.445],[302407.012,2778152.294],[302406.828,2778152.801],[302406.796,2778153.263],[302406.849,2778153.964],[302406.703,2778173.548],[302404.413,2778175.076],[302400,2778178.021],[302393.972,2778182.043],[302389.391,2778186.144],[302386.535,2778189.503],[302384.879,2778195.38],[302384.879,2778195.38]]] } ,	"properties": {	"BILL_CODE": "10967113566136",	"URGENT_RCV_DATE": "20241108",	"URGENT_START_DATE": "20241120090000",	"URGENT_END_DATE": "20241120170000",	"URGENT_ADDRESS1

In [1]:
import requests
from requests import Response
import csv
import pandas
import os
import sqlite3

In [ ]:
from pyproj import Transformer

# 創建轉換器
transformer = Transformer.from_crs("epsg:3826", "epsg:4326")
twd97_x = 305689.109
twd97_y = 2768207.525
# 轉換 TWD97 到 WGS84
latitude,longitude = transformer.transform(twd97_x, twd97_y)

print(f'TWD97轉WGS84: Latitude: {latitude:.7}, Longitude: {longitude:.7}')


TWD97轉WGS84: Latitude: 25.02091, Longitude: 121.5518


twd97是台灣的座標系統
WGS84是世界的座標系統
經度 longitude
緯度 latitude
所以台灣的X = longitude, Y = latitude

In [14]:
from geopy.geocoders import Nominatim

# 初始化 Nominatim 反向地理編碼器
geolocator = Nominatim(user_agent='PROJ')

def get_address_from_coordinates(lat, lon):
    """ 根據經緯度獲取地址或道路名稱 """
    location = geolocator.reverse((lat, lon), language='zh-TW', exactly_one=True)
    if location:
        return location.address
    return "未找到地址"

# 範例：查詢經緯度 (台北市)
latitude = 25.02066
longitude = 121.5518

address = get_address_from_coordinates(latitude, longitude)
print(address)
print(type(address))
print(f"座標 ({latitude}, {longitude}) 對應的地址是：{address}")


7號, 樂業街118巷, 芳和里, 大安區, 六張犁, 臺北市, 106, 臺灣
<class 'str'>
座標 (25.02066, 121.5518) 對應的地址是：7號, 樂業街118巷, 芳和里, 大安區, 六張犁, 臺北市, 106, 臺灣


In [47]:
address = "7號, 樂業街118巷, 芳和里, 大安區, 六張犁, 臺北市, 106, 臺灣"
list_address = [item.strip() for item in address.split(",")]
list_address.reverse()
print(list_address)
dic_address = {} 


['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街118巷', '7號']


In [23]:
display(type(response_json))
# print(response_json["features"][0]["properties"])
response_json01 = response_json["features"][0]["properties"]
for k,v in response_json01.items():
    print(f"{k}:{v}")

dict

BILL_CODE:10967113535908
URGENT_RCV_DATE:20241017
URGENT_START_DATE:20241116000000
URGENT_END_DATE:20241116220000
URGENT_ADDRESS1:台北市樂業街108巷1至11號
X1:305689.109
Y1:2768179.406
URGENT_ADDRESS2:台北市樂業街116號圍牆及118巷2~8號
X2:305689.109
Y2:2768179.406


In [3]:
    
def download_data():
    '''
    下載資料 需要做驗證只寫入不在資料庫裏面的那筆
    '''
    conn = sqlite3.connect("./TPEroad.db")
    url = 'https://tpnco.blob.core.windows.net/blobfs/Rally/TodayUrgentCase.json'
    try:
        response = requests.get(url)
        response.raise_for_status()
        with conn:
            cursor = conn.cursor()
            cursor.execute('''
            CREATE TABLE IF NOT EXISTS records (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                Bill_code TEXT,
                RCVdate TEXT,
                Start_date TEXT,
                End_date TEXT,
                Address1 TEXT,
                X1 TEXT,
                Y1 TEXT,
                Address2 TEXT,
                X2 TEXT,
                Y2 TEXT,
                UNIQUE(Address1,RCVdate)
            )
            ''')
            print("Table 'records' created or already exists.")
    except Exception as e:
        print(e)

    else:
        with conn:
            cursor = conn.cursor()
            for i in response_json['features']:
                for k in i['properties']:
                    Bill_code:str = i['properties']["BILL_CODE"]
                    RCVdate = i['properties']["URGENT_RCV_DATE"]
                    Start_date = i['properties']["URGENT_START_DATE"]
                    End_date = i['properties']["URGENT_END_DATE"]
                    Address1 = i['properties']["URGENT_ADDRESS1"]
                    X1 = i['properties']["X1"]
                    Y1 = i['properties']["Y1"]
                    Address2 = i['properties']["URGENT_ADDRESS2"]
                    X2 = i['properties']["X2"]
                    Y2 = i['properties']["Y2"]
                    
                    cursor = conn.cursor()
                    sql = '''INSERT OR IGNORE INTO records(Bill_code, RCVdate, Start_date, End_date, Address1, X1, Y1, Address2, X2, Y2)
                                values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
                            '''
                    cursor.execute(sql,(Bill_code, RCVdate, Start_date, End_date, Address1, X1, Y1, Address2, X2, Y2))
download_data()

Table 'records' created or already exists.


現在有每一個x,y轉換成的地址 再把地址存成我用的資料表 再從資料表裡面提取我需要的區的位置 就可以做combobox的下拉式選單

In [4]:
import functoins
import sqlite3

conn = sqlite3.connect('TPEroad.db')
with conn:
    cursor = conn.cursor()

    sql = '''
        SELECT Address1,x1,y1 FROM records
    '''

    cursor.execute(sql)
    # 把地址跟x1y1座標存進一個字典裡面
    address = [ ]
    for i in cursor.fetchall():
        di = {}
        di["地址"] = i[0]
        di["x1"] = i[1]
        di["y1"] = i[2]
        address.append(di)
    cursor.close() #把cursor關掉 清出記憶體
    display(address)

[{'地址': '北投區建民路１６１之５號', 'x1': '302384.879', 'y1': '2778195.38'},
 {'地址': '大安區敦化南路二段６３巷６４號', 'x1': '305668.124', 'y1': '2769314.433'},
 {'地址': '大安區臥龍街３１６號', 'x1': '306518.028', 'y1': '2767751.599'},
 {'地址': '松山區延壽街３１９－３２１號', 'x1': '306666.638', 'y1': '2772177.367'},
 {'地址': '大安區安居街２４號', 'x1': '306017.325', 'y1': '2768229.119'},
 {'地址': '松山區八德路三段１３７號', 'x1': '306001.614', 'y1': '2771219.82'},
 {'地址': '北投區明德路104號至108號', 'x1': '302435.414', 'y1': '2778037.181'},
 {'地址': '大安區大安路一段１７６巷５號', 'x1': '305016.716', 'y1': '2769932.84'},
 {'地址': '大安區和平東路一段87號  張立文理補習班', 'x1': '302991.933', 'y1': '2768872.108'},
 {'地址': '大安區溫州街５８巷14號', 'x1': '303683.128', 'y1': '2768161.802'},
 {'地址': '士林區士東路２６６巷３弄１２之１號', 'x1': '304193.216', 'y1': '2778349.522'},
 {'地址': '大安區永康街４２號', 'x1': '303447.801', 'y1': '2769138.498'},
 {'地址': '內湖區成功路四段６１巷３２弄４號', 'x1': '310035.908', 'y1': '2774866.6'},
 {'地址': '北投區自強街67-６９號', 'x1': '301893.092', 'y1': '2778128.963'},
 {'地址': '台北市文山區忠順街ㄧ段9巷1號', 'x1': '306441.723', 'y1': '2764127

接下來要提取每個字典裏面的x1y1然後丟到我的latlonturn裡面轉成地址(['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街', '114號'])  
再for i in list if '區' in i sql='insert ? into records' execute(sql,(i))


In [15]:
import functoins
import sqlite3
import time
conn = sqlite3.connect("TPEroad.db")
cursor = conn.cursor()
alladdress = []
for i in address:
    ad = i['地址']
    fuad = functoins.latlonturn("reverse",alladdress,i['x1'],i['y1'])
    fullstreet = ''.join(fuad)
    lat,lon = functoins.xytransform(i['x1'],i['y1'])
    for i in fuad:
        if "區" in i :dist = i
    add_content_sql = '''
    UPDATE RECORDS
    SET 新地址 = ?, 行政區 = ?, Lat = ?, Lon = ?
    WHERE address1 = ?;
    '''
    add_column_sql1='''
    ALTER TABLE RECORDS 
    ADD COLUMN 新地址 TEXT
    '''
    add_column_sql2='''
    ALTER TABLE RECORDS 
    ADD COLUMN 行政區 TEXT
    '''
    add_column_sql3='''
    ALTER TABLE RECORDS 
    ADD COLUMN Lat TEXT
    '''    
    add_column_sql4='''
    ALTER TABLE RECORDS 
    ADD COLUMN Lon TEXT
    '''


    cursor.execute(add_content_sql,(fullstreet, dist, lat, lon, ad))
    time.sleep(5)
conn.commit()


1.執行反轉操作
2.TWD97轉WGS84: Latitude: 25.1112, Longitude: 121.5194
3.單次轉換結果['臺灣', '112', '臺北市', '蘭雅', '北投區', '裕民里', '建民路', '161之11號']
4.總清單:[['臺灣', '112', '臺北市', '蘭雅', '北投區', '裕民里', '建民路', '161之11號']]
2.TWD97轉WGS84: Latitude: 25.1112, Longitude: 121.5194
1.執行反轉操作
2.TWD97轉WGS84: Latitude: 25.03091, Longitude: 121.5516
3.單次轉換結果['臺灣', '106', '臺北市', '車罾', '大安區', '義安里', '敦化南路二段63巷', '56號']
4.總清單:[['臺灣', '112', '臺北市', '蘭雅', '北投區', '裕民里', '建民路', '161之11號'], ['臺灣', '106', '臺北市', '車罾', '大安區', '義安里', '敦化南路二段63巷', '56號']]
2.TWD97轉WGS84: Latitude: 25.03091, Longitude: 121.5516
1.執行反轉操作
2.TWD97轉WGS84: Latitude: 25.01677, Longitude: 121.56
3.單次轉換結果['臺灣', '10676', '臺北市', '六張犁', '大安區', '黎和里', '臥龍街']
4.總清單:[['臺灣', '112', '臺北市', '蘭雅', '北投區', '裕民里', '建民路', '161之11號'], ['臺灣', '106', '臺北市', '車罾', '大安區', '義安里', '敦化南路二段63巷', '56號'], ['臺灣', '10676', '臺北市', '六張犁', '大安區', '黎和里', '臥龍街']]
2.TWD97轉WGS84: Latitude: 25.01677, Longitude: 121.56
1.執行反轉操作
2.TWD97轉WGS84: Latitude: 25.05672, Longitude: 121.5616
3.單次轉換結果['臺灣'

In [14]:
import functoins
address = [{'地址': '台北市樂業街108巷1至11號', 'x1': '305689.109', 'y1': '2768179.406'},
 {'地址': '士林區大東路１７６號', 'x1': '302989.777', 'y1': '2776097.833'}]
alladdress = []
# for i in address:
#     ad = i['地址']
#     fuad = functoins.latlonturn("reverse",alladdress,i['x1'],i['y1'])
#     type(fuad)
# for i in fuad:
#     if "區" in i :dist = i
#     print(dist)

fuad = functoins.latlonturn('reverse', alladdress, address[0]["x1"], address[0]["y1"])
fullstreet = ''.join(fuad)
display(fullstreet)

1.執行反轉操作
2.TWD97轉WGS84: Latitude: 25.02066, Longitude: 121.5518
3.單次轉換結果['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街118巷', '7號']
4.總清單:[['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街118巷', '7號']]


'臺灣106臺北市六張犁大安區芳和里樂業街118巷7號'

In [6]:
import functoins
x = 25.0951685
y = 121.528571

print(functoins.get_address_from_coordinates(x,y))

中山北路五段, 福林里, 士林區, 石角, 臺北市, 11111, 臺灣
